CP-SAT: (10 sec, 1 threads) = no output, (5 sec, 8 threads) = feasible but probably not optimal 12

In [7]:
import networkx as nx
from ortools.sat.python import cp_model

# Generate random graph
n = 500
p = 0.5
G = nx.gnp_random_graph(n, p, seed=0)

# Create CP-SAT model
model = cp_model.CpModel()

# Decision variables: x[i] = 1 if node i is in the independent set
x = [model.NewBoolVar(f'x[{i}]') for i in range(n)]

# Constraints: for every edge (u, v), at most one endpoint can be selected
for u, v in G.edges():
    model.Add(x[u] + x[v] <= 1)

# Objective: maximize the size of the independent set
model.Maximize(sum(x))

# Solver with time limitation
solver = cp_model.CpSolver()
solver.parameters.max_time_in_seconds = 10
solver.parameters.num_search_workers = 16      # parallel threads
solver.parameters.log_search_progress = True  # verbose output

# Solve the model
status = solver.Solve(model)

# Extract and display the results
print('Number of threads: ', solver.parameters.num_search_workers)
print('Time limitation (secs):', solver.parameters.max_time_in_seconds)
if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    independent_set = [i for i in range(n) if solver.Value(x[i])]
    print(f"Solver status: {solver.StatusName(status)}")
    print(f"Independent set size: {len(independent_set)}")
    print("Sample of selected nodes:", independent_set[:20])
else:
    print("No solution found within the time limit.")


Number of threads:  16
Time limitation (secs): 10.0
Solver status: FEASIBLE
Independent set size: 12
Sample of selected nodes: [20, 123, 169, 195, 247, 266, 269, 321, 322, 352, 371, 424]


Multi Testing

In [17]:
import time
import networkx as nx
from ortools.sat.python import cp_model

def solve_mis(n, p, seed, time_limit=10, num_workers=16):
    # 1) Generate graph
    G = nx.gnp_random_graph(n, p, seed=seed)
    
    # 2) Build model
    model = cp_model.CpModel()
    x = [model.NewBoolVar(f'x[{i}]') for i in range(n)]
    for u, v in G.edges():
        model.Add(x[u] + x[v] <= 1)
    model.Maximize(sum(x))
    
    # 3) Configure solver
    solver = cp_model.CpSolver()
    solver.parameters.max_time_in_seconds = time_limit
    solver.parameters.num_search_workers = num_workers
    solver.parameters.log_search_progress = False  # turn off per-node logs
    
    # 4) Solve and time it
    start = time.time()
    status = solver.Solve(model)
    elapsed = time.time() - start
    
    # 5) Extract result
    if status in (cp_model.OPTIMAL, cp_model.FEASIBLE):
        size = sum(1 for i in range(n) if solver.Value(x[i]))
    else:
        size = None
    
    return {
        'seed': seed,
        'status': solver.StatusName(status),
        'size': size,
        'time_sec': elapsed
    }


n = 2000
p = 0.5
time_limit = 20     # seconds per instance
workers   = 16      # parallel threads
    
results = []
for seed in range(10):
    print(f"Solving seed={seed}...", end=' ', flush=True)
    res = solve_mis(n, p, seed, time_limit, workers)
    print(f"{res['status']}, size={res['size']}, time={res['time_sec']:.1f}s")
    results.append(res)
    
# Summary
sizes = [r['size'] for r in results if r['size'] is not None]
avg_size = sum(sizes) / len(sizes) if sizes else 0
print("\nPer-seed results: ER(", n, ',', p, ') with time limit ', time_limit)
for r in results:
    print(f"  seed={r['seed']:>2}  status={r['status']:<8}  size={r['size']!s:<3}  time={r['time_sec']:.1f}s")
print(f"\nAverage independent set size (over feasible runs): {avg_size:.2f}")


Solving seed=0... UNKNOWN, size=None, time=23.3s
Solving seed=1... UNKNOWN, size=None, time=23.3s
Solving seed=2... UNKNOWN, size=None, time=23.9s
Solving seed=3... UNKNOWN, size=None, time=23.4s
Solving seed=4... UNKNOWN, size=None, time=23.9s
Solving seed=5... UNKNOWN, size=None, time=25.8s
Solving seed=6... UNKNOWN, size=None, time=25.3s
Solving seed=7... UNKNOWN, size=None, time=23.8s
Solving seed=8... UNKNOWN, size=None, time=23.5s
Solving seed=9... UNKNOWN, size=None, time=24.3s

Per-seed results: ER( 2000 , 0.5 ) with time limit  20
  seed= 0  status=UNKNOWN   size=None  time=23.3s
  seed= 1  status=UNKNOWN   size=None  time=23.3s
  seed= 2  status=UNKNOWN   size=None  time=23.9s
  seed= 3  status=UNKNOWN   size=None  time=23.4s
  seed= 4  status=UNKNOWN   size=None  time=23.9s
  seed= 5  status=UNKNOWN   size=None  time=25.8s
  seed= 6  status=UNKNOWN   size=None  time=25.3s
  seed= 7  status=UNKNOWN   size=None  time=23.8s
  seed= 8  status=UNKNOWN   size=None  time=23.5s
  se

Gurobi

In [3]:
import networkx as nx
import torch
import gurobipy as gp
from gurobipy import GRB

# 1) Generate the graph
n = 500
p = 0.5
G = nx.gnp_random_graph(n, p, seed=0)
A = torch.tensor(nx.adjacency_matrix(G).todense(), dtype=torch.int8)

# 2) Create Gurobi model
model = gp.Model("max_independent_set")

# 3) Add binary decision variables x[i] = 1 if node i is in the independent set
x = model.addVars(n, vtype=GRB.BINARY, name="x")

# 4) Add edge constraints: for each edge (u,v), x[u] + x[v] ≤ 1
for u, v in G.edges():
    model.addConstr(x[u] + x[v] <= 1)

# 5) Objective: maximize the size of the independent set
model.setObjective(gp.quicksum(x[i] for i in range(n)), GRB.MAXIMIZE)

# 6) Set a 10-second time limit
model.Params.TimeLimit = 10

# 7) (Optional) Suppress logging if you like:
# model.Params.OutputFlag = 0

# 8) Optimize
model.optimize()

# 9) Extract and print the solution
status = model.Status
if status in (GRB.OPTIMAL, GRB.TIME_LIMIT):
    sel = [i for i in range(n) if x[i].X > 0.5]
    # Map status code to a string
    status_str = (
        "OPTIMAL" if status == GRB.OPTIMAL
        else "TIME_LIMIT" if status == GRB.TIME_LIMIT
        else str(status)
    )
    print(f"Status: {status_str}")
    print(f"Independent set size: {len(sel)}")
    print("First 20 nodes in set:", sel[:20])
else:
    print("No feasible solution found. Status code:", status)



Set parameter TimeLimit to value 10
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 11.0 (26100.2))

CPU model: Intel(R) Core(TM) i9-10880H CPU @ 2.30GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Non-default parameters:
TimeLimit  10

Optimize a model with 62475 rows, 500 columns and 124950 nonzeros
Model fingerprint: 0xb9ab9743
Variable types: 0 continuous, 500 integer (500 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 6.0000000
Presolve removed 56487 rows and 0 columns
Presolve time: 1.35s
Presolved: 5988 rows, 500 columns, 40994 nonzeros
Variable types: 0 continuous, 500 integer (500 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
    3652    5.4698333e+01   1.276285e+03   0.000000e+00      5s
  

Multi Testing

In [24]:
# %% [markdown]
# Maximum Independent Set via Gurobi in VSCode Cells
# This notebook is organized in cells using the `# %%` marker for VSCode.

# %%
import time
import networkx as nx
import torch
import gurobipy as gp
from gurobipy import GRB

def solve_mis_gurobi(n, p, seed, time_limit=10, verbose=False):
    """Builds and solves MIS on G(n,p) using Gurobi, returns result dict."""
    # Generate graph
    G = nx.gnp_random_graph(n, p, seed=seed)
    # Create Gurobi model
    model = gp.Model(f"mis_seed_{seed}")
    # Add binary variables
    x = model.addVars(n, vtype=GRB.BINARY, name="x")
    # Edge constraints
    for u, v in G.edges():
        model.addConstr(x[u] + x[v] <= 1)
    # Objective
    model.setObjective(gp.quicksum(x[i] for i in range(n)), GRB.MAXIMIZE)
    # Parameters
    model.Params.TimeLimit = time_limit
    if not verbose:
        model.Params.OutputFlag = 0
    # Solve
    start = time.time()
    model.optimize()
    elapsed = time.time() - start
    # Extract result
    status = model.Status
    size = None
    if status in (GRB.OPTIMAL, GRB.TIME_LIMIT):
        size = sum(1 for i in range(n) if x[i].X > 0.5)
    status_str = (
        "OPTIMAL" if status == GRB.OPTIMAL
        else "TIME_LIMIT" if status == GRB.TIME_LIMIT
        else str(status)
    )
    return {
        'seed': seed,
        'status': status_str,
        'size': size,
        'time_sec': elapsed
    }

# %%
# Batch execution over seeds 0–9
n = 2000
p = 0.5
time_limit = 20  # seconds per instance
verbose = False

results = []
for seed in range(10):
    print(f"Solving seed={seed}...", end=' ', flush=True)
    res = solve_mis_gurobi(n, p, seed, time_limit, verbose)
    print(f"{res['status']}, size={res['size']}, time={res['time_sec']:.1f}s")
    results.append(res)

# %%
# Summary statistics
sizes = [r['size'] for r in results if r['size'] is not None]
avg_size = sum(sizes) / len(sizes) if sizes else 0

# print("\nPer-seed results:")
print("\nPer-seed results: ER(", n, ',', p, ') with time limit ', time_limit)
for r in results:
    print(f"  seed={r['seed']:>2}  status={r['status']:<10}  size={r['size']!s:<3}  time={r['time_sec']:.1f}s")

print(f"\nAverage independent set size (over feasible runs): {avg_size:.2f}")


Solving seed=0... Set parameter TimeLimit to value 20
TIME_LIMIT, size=10, time=20.4s
Solving seed=1... Set parameter TimeLimit to value 20
TIME_LIMIT, size=11, time=20.5s
Solving seed=2... Set parameter TimeLimit to value 20
TIME_LIMIT, size=11, time=20.4s
Solving seed=3... Set parameter TimeLimit to value 20
TIME_LIMIT, size=9, time=20.5s
Solving seed=4... Set parameter TimeLimit to value 20
TIME_LIMIT, size=11, time=20.5s
Solving seed=5... Set parameter TimeLimit to value 20
TIME_LIMIT, size=11, time=20.4s
Solving seed=6... Set parameter TimeLimit to value 20
TIME_LIMIT, size=10, time=20.4s
Solving seed=7... Set parameter TimeLimit to value 20
TIME_LIMIT, size=11, time=20.4s
Solving seed=8... Set parameter TimeLimit to value 20
TIME_LIMIT, size=11, time=20.5s
Solving seed=9... Set parameter TimeLimit to value 20
TIME_LIMIT, size=9, time=20.4s

Per-seed results: ER( 2000 , 0.5 ) with time limit  20
  seed= 0  status=TIME_LIMIT  size=10   time=20.4s
  seed= 1  status=TIME_LIMIT  size=